In [1]:
import sys
sys.executable

'D:\\PycharmProjects\\Job-Recommender\\venv\\Scripts\\python.exe'

In [2]:
import re, json, logging, torch
from typing import List, Dict, Any, Optional
from transformers import AutoTokenizer, AutoModelForCausalLM

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("skills-transformers")

JSON_ARRAY_RE = re.compile(r"(\[.*?\])", re.DOTALL)
model_id = "D:\PycharmProjects\Job-Recommender\model\qwen1.7b"


tok = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map=  "cuda", torch_dtype=torch.float16, trust_remote_code=True)
model.eval()


D:\PycharmProjects\Job-Recommender\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.26s/it]


Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=2048, out_features=6144, bias=False)
          (up_proj): Linear(in_features=2048, out_features=6144, bias=False)
          (down_proj): Linear(in_features=6144, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen3RMSNorm((2048,), eps=1e-06)
        (post_attention_layer

In [3]:
import torch
torch.cuda.empty_cache()


In [25]:

PROMPT = (
    "Extract ONLY atomic skill names from the resume below.\n"
    "Follow these strict rules:\n"
    "1. Each skill must be SHORT (1–3 words max).\n"
    "2. NO sentences.\n"
    "3. NO job duties.\n"
    "4. NO categories (e.g., 'Tools & Libraries', 'Computer Vision').\n"
    "5. ONLY proper skill names: software, tools, libraries, techniques, models.\n"
    "6. REMOVE duplicates.\n"
    "7. Output ONLY a JSON list of strings.\n"
    "8. No explanations. No extra text.\n\n"
    "Resume:\n\"\"\"\n{resume}\n\"\"\"\n\n"
    "/no_think"
)


PROMPT2 = """Extract ONLY atomic skill names, Tools, technique, certifications and Libraries from the resume below.
Rules:
1) Each skill must be SHORT (1–3 words).
2) NO sentences or job duties.
3) ONLY proper skill names: software, libraries, tools, techniques, models.
4) REMOVE duplicates.
4) Output ONLY a JSON list of strings (e.g. ["PyTorch","NumPy"]).
6) No extra text or explanation.

### EXAMPLES
Input: "Technical: PyTorch, TensorFlow, data preprocessing and model training in production"
Output: ["PyTorch","TensorFlow","Data preprocessing"]

Input: "Tools: MySQL/MongoDB, Flask/Django, Git"
Output: ["MySQL","MongoDB","Flask","Django","Git"]

Resume:
\"\"\"\n{resume}\n\"\"\"\n
Answer (JSON only):"""



PROMPT3 = """
Extract ONLY atomic skill names, tools, techniques, certifications, and libraries from the resume below.

Additionally:
- Detect the TOTAL years of professional experience of the person. Strictly don't consider the Education in that.
- If mentioed explicitely, that is good, but rare, so you need to find the internship / work experience and add up the timeline to get the final years of experience.
- Return ONLY the numeric value (integer or float) for years of experience.
- Append that numeric value as the LAST element in the JSON list you return.

Rules:
1) Each skill must be SHORT (1–3 words).
2) NO sentences, no job duties, no descriptions.
3) ONLY real skill names: software, libraries, frameworks, tools, techniques, models, cloud platforms, certifications.
4) REMOVE duplicates.
5) If the resume implies a range (e.g., “3–5 years”), take the upper value.
6) If multiple jobs exist, SUM the years across roles unless clearly overlapping.
7) If unclear, infer a reasonable estimate from the career timeline.
8) Output ONLY a JSON list of strings, example: ["PyTorch","TensorFlow","Git",3.5].
9) NO extra text or explanation.

### EXAMPLES

Input: "Technical: PyTorch, TensorFlow, data preprocessing and model training in production"
Output: ["PyTorch","TensorFlow","Data preprocessing",2]

Input: "Tools: MySQL/MongoDB, Flask/Django, Git"
Output: ["MySQL","MongoDB","Flask","Django","Git",1]

Input: "Experience: Worked 2019–2023 as ML Engineer"
Output: ["ML Engineering",4]

---

Resume:
\"\"\"\n{resume}\n\"\"\"\n

Answer (JSON only):
"""


In [26]:
import ast
import json
import re
from typing import Optional, Tuple, Any

def extract_list(text: str, debug: bool = False) -> Optional[Any]:
    """
    Robustly extract the first list-like object found *after* a </think> tag.
    If no </think> is present, falls back to the first top-level list in the string.
    Returns:
      - a Python list on success
      - None on failure (or (None, diagnostics) if debug=True)
    Debug info is returned as a dict when debug=True.
    """
    diagnostics = {"original_len": None, "sanitized_len": None, "tag_found": False,
                   "start_idx": None, "end_idx": None, "list_str_preview": None,
                   "ast_ok": False, "json_ok": False, "error": None, "method": None}

    if not isinstance(text, str):
        diagnostics["error"] = "input not a str"
        return (None, diagnostics) if debug else None

    diagnostics["original_len"] = len(text)

    # --- sanitize: remove control chars except \n\r\t, normalize curly quotes ---
    text = "".join(ch for ch in text if (ord(ch) >= 32 or ch in ("\n", "\r", "\t")))
    text = text.replace("\u201c", '"').replace("\u201d", '"').replace("\u2018", "'").replace("\u2019", "'")
    diagnostics["sanitized_len"] = len(text)

    # strip outer matching quotes if whole string is quoted
    if (text.startswith("'") and text.endswith("'")) or (text.startswith('"') and text.endswith('"')):
        text = text[1:-1]

    # try find </think> (case-insensitive). If found, search after it; else search entire string
    m = re.search(r"</think\s*>", text, flags=re.IGNORECASE)
    search_start = m.end() if m else 0
    diagnostics["tag_found"] = bool(m)

    # helper: find first top-level list starting at pos
    def find_top_level_list(s: str, pos: int = 0) -> Tuple[Optional[int], Optional[int]]:
        start = s.find('[', pos)
        if start == -1:
            return None, None
        depth = 0
        in_string = False
        string_char = None
        escape = False
        for i in range(start, len(s)):
            ch = s[i]
            if escape:
                escape = False
                continue
            if in_string:
                if ch == '\\':
                    escape = True
                    continue
                if ch == string_char:
                    in_string = False
                    string_char = None
                continue
            else:
                if ch == '"' or ch == "'":
                    in_string = True
                    string_char = ch
                    continue
                if ch == '[':
                    depth += 1
                    continue
                if ch == ']':
                    depth -= 1
                    if depth == 0:
                        return start, i
                    continue
        return None, None

    start, end = find_top_level_list(text, search_start)

    # fallback: if not found after </think>, try from beginning
    if start is None and search_start != 0:
        start, end = find_top_level_list(text, 0)

    if start is None:
        diagnostics["error"] = "no top-level list found"
        return (None, diagnostics) if debug else None

    diagnostics["start_idx"] = start
    diagnostics["end_idx"] = end
    list_str = text[start:end+1]
    diagnostics["list_str_preview"] = (list_str[:200] + "...") if len(list_str) > 200 else list_str

    # Try safe Python literal parsing first
    try:
        parsed = ast.literal_eval(list_str)
        if isinstance(parsed, list):
            diagnostics["ast_ok"] = True
            diagnostics["method"] = "ast"
            return (parsed, diagnostics) if debug else parsed
        else:
            diagnostics["error"] = "ast parsed, result not a list"
    except Exception as e_ast:
        diagnostics["error"] = f"ast failed: {e_ast}"

    # Try JSON (requires double quotes)
    try:
        parsed_json = json.loads(list_str)
        if isinstance(parsed_json, list):
            diagnostics["json_ok"] = True
            diagnostics["method"] = "json"
            return (parsed_json, diagnostics) if debug else parsed_json
        else:
            diagnostics["error"] = "json parsed, result not a list"
    except Exception as e_json:
        diagnostics["error"] = f"json failed: {e_json}"

    # Last-ditch: try to coerce single quotes to double quotes safely for JSON
    # Replace only unescaped single quotes that are list/string delimiters (naive but practical)
    try:
        # avoid touching escaped quotes
        coerced = re.sub(r"(?<!\\)'", '"', list_str)
        parsed_coerced = json.loads(coerced)
        if isinstance(parsed_coerced, list):
            diagnostics["json_ok"] = True
            diagnostics["method"] = "json_coerced"
            diagnostics["note"] = "coerced single->double quotes"
            return (parsed_coerced, diagnostics) if debug else parsed_coerced
        else:
            diagnostics["error"] = "coerced json parsed, result not a list"
    except Exception as e_coerce:
        diagnostics["error"] = f"coerced json failed: {e_coerce}"

    return (None, diagnostics) if debug else None


In [30]:
from typing import Tuple, Optional
from transformers import StoppingCriteria, StoppingCriteriaList

def extract_skills_transformers(resume_text: str, model_id: str):
    """
    Minimal extractor that DISABLES 'thinking' by stopping generation after the first ']'.
    Returns: (decoded_generated_text, parsed_json_or_None, filled_offsets_list).
    Reuses global `tok` and `mdl` if present; otherwise loads them simply.
    """
    # Local stopping criterion that halts when a closing bracket appears
    class StopOnFirstClosingBracket(StoppingCriteria):
        def __init__(self, tokenizer):
            self.tokenizer = tokenizer
        def __call__(self, input_ids, scores) -> bool:
            # decode current tokens and stop if we see ']'
            try:
                text = self.tokenizer.decode(input_ids[0], skip_special_tokens=True)
                return text.count("]") >= 1
            except Exception:
                return False

    # reuse globals if available
    try:
        tokenizer = tok  # type: ignore[name-defined]
        # model = mdl      # type: ignore[name-defined]
        global model
    except Exception:
        # minimal load if globals missing
        tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
        
         # choose device
        device = "cuda" if torch.cuda.is_available() else "cpu"

        if device == "cuda":
            # prefer fp16 on CUDA for speed & memory
            model = AutoModelForCausalLM.from_pretrained(
                model_id,
                torch_dtype=torch.float16,
                trust_remote_code=True,
            )
            model.to(device)
        else:
            model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True)
            model.to(device)

        model.eval()

    # prepare prompt and tokenized inputs
    prompt = PROMPT2.format(resume=resume_text)
    # inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=tokenizer.model_max_length).to(model.device)
    # input_ids = inputs["input_ids"]
    # input_len = input_ids.shape[1]

    messages = [{"role": "user", "content": prompt + "/no_think"}] # + "/no_think"

    inputs = tokenizer.apply_chat_template(
        messages,
        return_tensors="pt",
        add_generation_prompt=True
    ).to(model.device)
    print("inputs: ", inputs)

    input_ids = inputs
    input_len = input_ids.shape[1]


    # generate with stopping criterion (stops immediately after first ']')
    stopping = StoppingCriteriaList([StopOnFirstClosingBracket(tokenizer)])
    with torch.no_grad():
        out = model.generate(
            inputs,
            max_new_tokens=1024,
            do_sample=False,
            return_dict_in_generate=True,
            output_scores=False,
            # stopping_criteria=stopping,
        )

    decoded = tokenizer.decode(out.sequences[0][input_len:], skip_special_tokens=True)
    print("raw output: ",decoded)
    decoded = extract_list(decoded)
    print("Generation completed: " , decoded)
    
    return decoded


In [31]:
output = extract_skills_transformers(resume2, "D:\PycharmProjects\Job-Recommender\model\qwen1.7b")

inputs:  tensor([[151644,    872,    198,  ..., 151644,  77091,    198]],
       device='cuda:0')
raw output:  <think>

</think>

["Deep Learning", "NLP", "Computer Vision & Image Processing", "Tools", "Libraries", "Databricks", "PowerBI", "Tableau", "MLflow", "OpenCV", "NLTK", "SpaCy", "NumPy", "Pandas", "Matplotlib/Seaborn", "Scikit-learn", "MySQL", "MongoDB", "Pinecone", "FAISS", "Chroma", "Flask", "Django", "Git", "Evidently AI", "WEKA", "LangChain", "LangGraph", "RAG", "Vector Databases", "Prompting Techniques", "GANs", "Autoencoders", "Model Monitoring (MLOps: Model Health Analysis)", "ONNX", "GNNs", "Neural Networks (CNNs, RNNs, Transformers)", "Explainable AI (XAI)", "FP-Growth", "PCA", "SVMs", "ensemble techniques", "LIME", "SHAP", "Auto Number Plate Detection and Recognition", "Concrete Compressive Strength Prediction", "Model Monitoring (MLOps)", "Model Health Analysis", "Model Retraining", "Model Interpretability", "Image Classification", "Object Detection", "Image Segmenta

In [32]:
output

['Deep Learning',
 'NLP',
 'Computer Vision & Image Processing',
 'Tools',
 'Libraries',
 'Databricks',
 'PowerBI',
 'Tableau',
 'MLflow',
 'OpenCV',
 'NLTK',
 'SpaCy',
 'NumPy',
 'Pandas',
 'Matplotlib/Seaborn',
 'Scikit-learn',
 'MySQL',
 'MongoDB',
 'Pinecone',
 'FAISS',
 'Chroma',
 'Flask',
 'Django',
 'Git',
 'Evidently AI',
 'WEKA',
 'LangChain',
 'LangGraph',
 'RAG',
 'Vector Databases',
 'Prompting Techniques',
 'GANs',
 'Autoencoders',
 'Model Monitoring (MLOps: Model Health Analysis)',
 'ONNX',
 'GNNs',
 'Neural Networks (CNNs, RNNs, Transformers)',
 'Explainable AI (XAI)',
 'FP-Growth',
 'PCA',
 'SVMs',
 'ensemble techniques',
 'LIME',
 'SHAP',
 'Auto Number Plate Detection and Recognition',
 'Concrete Compressive Strength Prediction',
 'Model Monitoring (MLOps)',
 'Model Health Analysis',
 'Model Retraining',
 'Model Interpretability',
 'Image Classification',
 'Object Detection',
 'Image Segmentation',
 'Text Preprocessing',
 'Text Classification',
 'NER',
 'Text Summariza

In [20]:
# prompt = PROMPT2.format(resume=resume4)
# messages = [{"role": "user", "content": prompt + "/no_think"}]
# inputs = tok.apply_chat_template(
#     messages,
#     return_tensors="pt",
#     add_generation_prompt=True
# ).to(model.device)
# input_len = inputs.shape[1]
# print(input_len)


# print(tok.decode(raw.sequences[0][input_len:], skip_special_tokens=True))

In [ ]:
# raw[0]

In [8]:

resume = "Skills: Python, Docker. Experience: used Mastercam."
resume2 = """
Yatrikkumar Shah
Kaggle Expert | Generative AI | Agentic AI | Data Science | Artificial Intelligence
+1 585-685-9681 | yatriksshah@gmail.com | linkedin.com/in/yatrikkumar-shah | github.com/Yatrik07
Education
University of Houston, Houston, USA August 2024 - May 2026
MS | Engineering Data Science | GPA: 3.85/4.0
Gujarat University, Ahmedabad, India July 2019 - April 2023
Honors BS | Artificial Intelligence and Machine Learning | GPA: 3.75/4.0
Technical Skills
Deep Learning: Agentic AI, LangChain, LangGraph, RAG, Vector Databases, Prompting Techniques, GANs, Autoencoders,
 Model Monitoring (MLOps: Model Health Analysis), ONNX, GNNs, Neural Networks (CNNs, RNNs, Transformers)
NLP: Text Preprocessing, Text Classification, NER, Text Summarization, Language Modeling (Seq2Seq, Transformers), LLMs
Computer Vision & Image Processing: Object Tracking, Image Classification, Object Detection, Image Segmentation
Tools & Libraries: Databricks, PowerBI, Tableau, MLflow, OpenCV, NLTK, SpaCy, NumPy, Pandas, Matplotlib/Seaborn, Scikit-
 learn, Databases – MySQL/ MongoDB, Vector Databases – Pinecone/FAISS/Chroma, Flask, Django, Git, Evidently AI, WEKA
Work Experience
University of Houston, Teaching Assistant January 2025 - Present
 • Teaching Assistant for graduate course “Data Mining for Engineers”, supporting labs, grading, and advanced topics
 including FP-Growth, PCA, SVMs, ensemble techniques and practical concepts like LangChain, PyTorch
Cloudoffis Technologies LLP, AI Engineer May 2023 - June 2024
 • Fine-tuned language models to classify financial document types into over 70 categories based on business requirements
 • Deployed LLaMA-13B for Document Information Retrieval based on Retrieval-Augmented Generation (RAG)
 • Implemented Automated Model Monitoring (MLOps) pipeline using statistical model health metrics and alert system for
 models whose health dropped below 40% followed by model retraining
 • Applied Explainable AI (XAI) techniques such as LIME and SHAP to interpret segment-level contributions in images
 (CNNs), documents (ML classifiers), and sequence models (RNNs) improving model interpretability
iNeuron, Deep Learning Intern October 2022 - January 2023
 • Engineered and deployed a deep learning system for Automatic Number Plate Detection and Recognition achieving 95%
 accuracy in real-world conditions, significantly enhancing road safety and security through real-time vehicle monitoring
 • Implemented advanced image processing techniques and state-of-the-art computer vision models that improved
 accuracy under varied environmental conditions, achieving a 25% increase in performance
iNeuron, Machine Learning Intern July 2022 - September 2022
 • Developed an end-to-end ready for business usage machine learning solution for predicting concrete compressive
 strength, which significantly reduced the testing time from 28 days to immediate model-based predictions
 • Designed a retraining-deployment pipeline to sustain model accuracy and ensure application reliability
 • Utilized clustering techniques to optimize supervised model performance in the pipeline to manage complex datasets
Achievements
• 1st Place Winner at Ingenious Hackathon 5.0 awarded a cash prize of $2000 and a technical course ($1200) at
 Ahmedabad University for an AI-based Disaster Management and Rescue project
• 1st rank in the Presentation at Arya-2023, National Applied Mathematics Conference for "Interview Analysis"
Projects
Easy-AI (Open-Source Project Lead) January 2025 - Present
 • Leading an intuitive Auto-AI Python library development, simplifying AI model creation for non-technical users through
 no-code interfaces and modular UI components
 • Enabling end-to-end workflows for regression, classification, clustering, image classification/segmentation, text
 classification, seq2seq language models, and QA chatbots
 • Designed training pipelines and model interpretation tools, currently preparing for deployment as “pip install easy-ai”
Resume Parsing and Interview Analysis Tool July 2024 - October 2024
 • Automated resume parsing with a custom NER model, effectively extracting candidate information, with 95% accuracy
 • Formulated a custom ranking algorithm that matched resumes with job descriptions, improving hiring efficiency
 • Analyzed interview videos using computer vision techniques to enhance candidate behavior and engagement analysis
 • Leveraged Whisper, MediaPipe, Librosa and ANN models to streamline video analysis, reducing human bias in recruitment
First Response: Disaster Management Rescue App March 2024 - July 2024
 • Developed first-of-its-kind disaster rescue app leveraging YOLOv8 and DeepSORT models to detect, track, and classify
 individuals as wounded or normal in disaster zones using live Drone Footage
"""


resume3 = """
"         HR ADMINISTRATOR/MARKETING ASSOCIATE

HR ADMINISTRATOR       Summary     Dedicated Customer Service Manager with 15+ years of experience in Hospitality and Customer Service Management.   Respected builder and leader of customer-focused teams; strives to instill a shared, enthusiastic commitment to customer service.         Highlights         Focused on customer satisfaction  Team management  Marketing savvy  Conflict resolution techniques     Training and development  Skilled multi-tasker  Client relations specialist           Accomplishments      Missouri DOT Supervisor Training Certification  Certified by IHG in Customer Loyalty and Marketing by Segment   Hilton Worldwide General Manager Training Certification  Accomplished Trainer for cross server hospitality systems such as    Hilton OnQ  ,   Micros    Opera PMS   , Fidelio    OPERA    Reservation System (ORS) ,   Holidex    Completed courses and seminars in customer service, sales strategies, inventory control, loss prevention, safety, time management, leadership and performance assessment.        Experience      HR Administrator/Marketing Associate

HR Administrator     Dec 2013   to   Current      Company Name   ï¼   City  ,   State     Helps to develop policies, directs and coordinates activities such as employment, compensation, labor relations, benefits, training, and employee services.  Prepares employee separation notices and related documentation  Keeps records of benefits plans participation such as insurance and pension plan, personnel transactions such as hires, promotions, transfers, performance reviews, and terminations, and employee statistics for government reporting.  Advises management in appropriate resolution of employee relations issues.  Administers benefits programs such as life, health, dental, insurance, pension plans, vacation, sick leave, leave of absence, and employee assistance.     Marketing Associate Â    Designed and created marketing collateral for sales meetings, trade shows and company executives.  Managed the in-house advertising program consisting of print and media collateral pieces.  Assisted in the complete design and launch of the company's website in 2 months.  Created an official company page on Facebook to facilitate interaction with customers.  Analyzed ratings and programming features of competitors to evaluate the effectiveness of marketing strategies.         Advanced Medical Claims Analyst     Mar 2012   to   Dec 2013      Company Name   ï¼   City  ,   State     Reviewed medical bills for the accuracy of the treatments, tests, and hospital stays prior to sanctioning the claims.  Trained to interpret the codes (ICD-9, CPT) and terminology commonly used in medical billing to fully understand the paperwork that is submitted by healthcare providers.  Required to have organizational and analytical skills as well as computer skills, knowledge of medical terminology and procedures, statistics, billing standards, data analysis and laws regarding medical billing.         Assistant General Manager     Jun 2010   to   Dec 2010      Company Name   ï¼   City  ,   State     Performed duties including but not limited to, budgeting and financial management, accounting, human resources, payroll and purchasing.  Established and maintained close working relationships with all departments of the hotel to ensure maximum operation, productivity, morale and guest service.  Handled daily operations and reported directly to the corporate office.  Hired and trained staff on overall objectives and goals with an emphasis on high customer service.  Marketing and Advertising, working on public relations with the media, government and local businesses and Chamber of Commerce.         Executive Support / Marketing Assistant     Jul 2007   to   Jun 2010      Company Name   ï¼   City  ,   State     Provided assistance to various department heads - Executive, Marketing, Customer Service, Human Resources.  Managed front-end operations to ensure friendly and efficient transactions.  Ensured the swift resolution of customer issues to preserve customer loyalty while complying with company policies.  Exemplified the second-to-none customer service delivery in all interactions with customers and potential clients.         Reservation & Front Office Manager     Jun 2004   to   Jul 2007      Company Name   ï¼   City  ,   State          Owner/ Partner     Dec 2001   to   May 2004      Company Name   ï¼   City  ,   State          Price Integrity Coordinator     Aug 1999   to   Dec 2001      Company Name   ï¼   City  ,   State          Education      N/A  ,   Business Administration   1999     Jefferson College   ï¼   City  ,   State       Business Administration  Marketing / Advertising         High School Diploma  ,   College Prep. studies   1998     Sainte Genevieve Senior High   ï¼   City  ,   State       Awarded American Shrubel Leadership Scholarship to Jefferson College         Skills     Accounting, ads, advertising, analytical skills, benefits, billing, budgeting, clients, Customer Service, data analysis, delivery, documentation, employee relations, financial management, government relations, Human Resources, insurance, labor relations, layout, Marketing, marketing collateral, medical billing, medical terminology, office, organizational, payroll, performance reviews, personnel, policies, posters, presentations, public relations, purchasing, reporting, statistics, website.    "

"""

resume4 = """
Bhumik Shah
Information Security Analyst

+1 551 263 6391 | 98bhumik@gmail.com | bhumik98.github.io/

SUMMARY:
Cybersecurity & Privacy Analyst with 4+ years of experience in threat detection, incident response, and security posture improvement
across cloud (AWS, Azure) and network environments. Skilled in SIEM, Privacy, EDR, DevSecOps, Threat Intelligence, Vulnerability
Management, PenTesting, GRC, and SOX Compliance. Proven in optimizing SOC workflows, automating security processes, and
reducing enterprise risk. Proven track record of optimizing security workflows, mitigating risks, and enhancing enterprise security
resilience.
PROFESSIONAL EXPERIENCE:
Nurix Therapeutics | Security Analyst (Contract) May 2024 –Current
• Worked directly with CISO and lead regular forums with development teams to break down business requirements into
development activities with multiple teams including Clinical, Operations and Cybersecurity teams
• Performed audits & tests for existing risk and controls in general and application control environments, including information
security, IAM, Dev(sec)ops CI/CD, incident analysis, database security, OS analysis, logging & monitoring and change management
• Investigate and respond to incidents, including analyzing data logs, identifying vulnerabilities, and coordinating with other IT teams
to mitigate risks, monitored and administered threat detection and mitigation via a SIEM service (SentinelOne & Qualys)
• Implemented incident analysis using KQL in Microsoft defender and generating weekly reports to present in weekly CyberSec
meetings. Conducted utilization of data from Microsoft Intune for device management and security policy enforcement.
• Experience with SSO, Lifecycle Management, Universal Directory, and API Access Management. Successfully migrated SSO from
Okta to Entra for over 70+ apps and services, onboarded SSO applications using SAML and OpenID Connect protocols
• Built AWS security with Wiz policies and JIRA integration, cutting manual reviews by 70%, resolving 300+ misconfigs, reducing
risk by 25%, and enhancing cross-team collaboration.
• Assisted in privacy incident response planning and execution using OneTrust, streamlining the identification, containment,
notification, and remediation of data breaches in near real-time
• Utilized Microsoft Entra Insights, Intune analytics, and CyberCNS dashboards to monitor security posture, generate compliance
reports, perform data privacy and compliance reviews based on NIST & SOC frameworks and communicate findings to leadership
• Deployed and managed multifactor authentication (MFA), conditional access policies, and endpoint security baselines to meet CIS
requirements and enhance organizational security posture.
AK Consulting Services | Cloud Security Analyst (Volunteer) Jul 2023 – Jan 2024
• Managed Oracle cloud security development from concept to deployment, employing Agile and Waterfall methodologies, gathered
requirements and designed UML diagrams, including use cases and activity diagrams, using MS Visio.
• Refined and tokenized logs captured using Cloud Service Provider event monitoring APIs to generate alerts or block user access
through management API in case of critical security violations
JIM Solutions | Security & Privacy Analyst May 2019 – Jul 2021
• Involved in network penetration testing and proactive network defense; digital forensic investigations, evidence handling, and
internal phishing campaign development, execution, and reporting; and system administration of information security tools.
• Assessed vulnerabilities in applications, operating systems, cloud environments, leveraging tools like Nessus, Qualys, and Open VAS to
conduct regular security scans. Responsible for testing, system integration, conducting User Acceptance Testing (UAT)
• Correlated data from various security tools (EDR, IPS, honeypots) to gain comprehensive threat intelligence.
• Participated in the Privacy Incident Management on-call rotation, triaging alerts and Whitehat reports to quantify impact and
determine the scope of attacks, driving incidents to closure by creating incident response playbooks
• Supported plan and execution of multiple concurrent IT audits, including reviews of cyber security, existing production applications,
systems currently being developed, technology infrastructure and specialized or emerging technologies
TECHNICAL SKILLS:
Language: Python, R, SQL, SAS, C/C++, Bash
Compliance & Frameworks: ISO 27001, NIST 800, COBIT, GDPR, SOC 2, SOX, FedRAMP
Security: Network Security, Penetration Testing, MITRE ATT&CK Framework, Forensics, OWASP Top 10, Vulnerability Assessment,
Access Management, IAM Tools, DNS, HTTP, TLS, GDPR, Cyberlaw, Applied Cryptography, Forensics
Certifications: Certified AWS Solution Architect, Certified Tableau Desktop Specialist, Microsoft Certified Security Operation Analyst
[SC-200], Certified Information Systems Auditor (CISA){Ontrack}, Certified Information Privacy Professional (CIPP/E) {Ontrack}
EDUCATION:
Master of Science in Information Systems | Stevens Institute of Technology [GPA: 3.9/4]
Bachelor of Engineering in Computer Engineering | University of Mumbai
"""


In [30]:
from transformers import pipeline

model_name_or_path = "Qwen/Qwen3-8B"

generator = pipeline(
    "text-generation", 
    model_name_or_path, 
    torch_dtype="auto", 
    device_map="auto",
)


D:\PycharmProjects\Job-Recommender\venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shahy\.cache\huggingface\hub\models--Qwen--Qwen3-8B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
`torch_dtype` is deprecated! Use `dtype` instead!
Fetching 5 files:   0%|                                          

In [35]:
import pandas as pd
tp = pd.read_csv("dataset/jd/job_descriptions.csv")
tp

,Job Id,Experience,Qualifications,Salary Range,location,Country,latitude,longitude,Work Type,Company Size,...,Contact,Job Title,Role,Job Portal,Job Description,Benefits,skills,Responsibilities,Company,Company Profile
0,1089843540111562,5 to 15 Years,M.Tech,$59K-$99K,Douglas,Isle of Man,54.2361,-4.5481,Intern,26801,...,001-381-930-7517x737,Digital Marketing Specialist,Social Media Manager,Snagajob,Social Media Managers oversee an organizations...,"{'Flexible Spending Accounts (FSAs), Relocatio...","Social media platforms (e.g., Facebook, Twitte...","Manage and grow social media accounts, create ...",Icahn Enterprises,"{""Sector"":""Diversified"",""Industry"":""Diversifie..."
1,398454096642776,2 to 12 Years,BCA,$56K-$116K,Ashgabat,Turkmenistan,38.9697,59.5563,Intern,100340,...,461-509-4216,Web Developer,Frontend Web Developer,Idealist,Frontend Web Developers design and implement u...,"{'Health Insurance, Retirement Plans, Paid Tim...","HTML, CSS, JavaScript Frontend frameworks (e.g...","Design and code user interfaces for websites, ...",PNC Financial Services Group,"{""Sector"":""Financial Services"",""Industry"":""Com..."
2,481640072963533,0 to 12 Years,PhD,$61K-$104K,Macao,"Macao SAR, China",22.1987,113.5439,Temporary,84525,...,9687619505,Operations Manager,Quality Control Manager,Jobs2Careers,Quality Control Managers establish and enforce...,"{'Legal Assistance, Bonuses and Incentive Prog...",Quality control processes and methodologies St...,Establish and enforce quality control standard...,United Services Automobile Assn.,"{""Sector"":""Insurance"",""Industry"":""Insurance: P..."
3,688192671473044,4 to 11 Years,PhD,$65K-$91K,Porto-Novo,Benin,9.3077,2.3158,Full-Time,129896,...,+1-820-643-5431x47576,Network Engineer,Wireless Network Engineer,FlexJobs,"Wireless Network Engineers design, implement, ...","{'Transportation Benefits, Professional Develo...",Wireless network design and architecture Wi-Fi...,"Design, configure, and optimize wireless netwo...",Hess,"{""Sector"":""Energy"",""Industry"":""Mining, Crude-O..."
4,117057806156508,1 to 12 Years,MBA,$64K-$87K,Santiago,Chile,-35.6751,-71.5429,Intern,53944,...,343.975.4702x9340,Event Manager,Conference Manager,Jobs2Careers,A Conference Manager coordinates and manages c...,"{'Flexible Spending Accounts (FSAs), Relocatio...",Event planning Conference logistics Budget man...,Specialize in conference and convention planni...,Cairn Energy,"{""Sector"":""Energy"",""Industry"":""Energy - Oil & ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1615935,134563577088850,0 to 12 Years,B.Tech,$64K-$114K,"Malabo (de jure),",Equatorial Guinea,1.6508,10.2679,Full-Time,18281,...,950-451-5843,Mechanical Engineer,Mechanical Design Engineer,ZipRecruiter,Mechanical Design Engineers create and develop...,"{'Employee Assistance Programs (EAP), Tuition ...","Mechanical engineering CAD software (e.g., Sol...","Design mechanical systems, components, and pro...",The Hershey Company,"{""Sector"":""Food and Beverage/Confectionery"",""I..."
1615936,618604818190827,2 to 14 Years,M.Tech,$62K-$130K,Warsaw,Poland,51.9194,19.1451,Intern,63621,...,676.387.1572x71877,IT Manager,IT Director,USAJOBS,An IT Director oversees an organizations IT de...,"{'Health Insurance, Retirement Plans, Paid Tim...",Strategic IT planning Leadership and managemen...,Provide strategic leadership for IT department...,EQT,"{""Sector"":""Energy"",""Industry"":""Energy"",""City"":..."
1615937,615471367712200,4 to 15 Years,BCA,$60K-$96K,Ashgabat,Turkmenistan,38.9697,59.5563,Part-Time,114287,...,537.384.6193x5284,Mechanical Engineer,Mechanical Design Engineer,Indeed,Mechanical Design Engineers create and develop...,"{'Tuition Reimbursement, Stock Options or Equi...","Mechanical engineering CAD software (e.g., Sol...","Design mechanical systems, components, and pro...",KLA,"{""Sector"":""Technology"",""Industry"":""Semiconduct..."
1615938,804137342023945,5 to 15 Years,BCA,$65K-$103K,Ouagadougou,Burk

In [37]:
tp.loc[0, "Job Description"]

'Social Media Managers oversee an organizations social media presence. They create and schedule content, engage with followers, and analyze social media metrics to drive brand awareness and engagement.'